# UTS

## Install library

In [ ]:
!pip install gensim
!pip instal tqdm
!pip instal nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.4 MB/s eta 0:00:00
  Created wheel for jcopml: filename=jcopml-1.2.2-py2.py3-none-any.whl size=32194 sha256=ec5040c4855c04f2233ec3b785bf5a88cadd3913a21013313743450bde64f7bb
  Stored in directory: /root/.cache/pip/wheels/a6/1c/91/32f7a6b0a9d6bbcef17a7fc0d065f9a9b8d9904551b377b64d
Successfully built jcopml
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: unknown command "instal" - maybe you meant "install"
ERROR: unknown command "instal" - maybe you meant "install"


In [ ]:
import os
import pandas as pd #pandas
import numpy as np #numpy
from tqdm.auto import tqdm
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words('indonesian'))

In [56]:
df = pd.read_csv('/content/drive/MyDrive/prosaindata/tugas/pta_fix.csv')

In [57]:
df

,stem,label
0,sistem informasi akademik siakad sistem inform...,rpl
1,berjalannya koneksi jaringan komput lancar gan...,rpl
2,web server perangkat lunak server berfungsimen...,rpl
3,seir perkembangan teknolog didunia muncul tekn...,komputasi
4,gerak pekerja game memiliki genr rt realtim st...,komputasi
...,...,...
591,investasi saham memiliki resiko kerugian dikar...,komputasi
592,inform retriev ir pengambilan informasi tersim...,pba
593,klasifikasi citra prose pengelompokan piksel c...,komputasi
594,identifikasi atribut pejalan kaki salah peneli...,komputasi


In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [60]:
sentences = [word_tokenize(kata) for kata in tqdm(df.stem)]

  0%|          | 0/596 [00:00<?, ?it/s]

## Train Word2Vec Model

In [ ]:
model_word2vec = Word2Vec(sentences, vector_size=128, window=5, min_count=3, workers=4, sg=0, hs=0, epochs=1000)

### Save Word2Vec Model

In [14]:
folder = 'w2v/'
path = '/content/drive/MyDrive/prosaindata/prosaindata/' + folder 
os.makedirs(path, exist_ok=True)

In [15]:
model_word2vec.save(path+'text_klasifikasi.w2v')

## Encoding

In [16]:
from gensim.models import Word2Vec

In [17]:
w2v = Word2Vec.load(path+'text_klasifikasi.w2v').wv

In [18]:
def sent_vector(sentence, w2v_model, stopword):
  vecs = [w2v_model[kata.lower()] for kata in (sentence) if kata in w2v_model]
  sent_vec = np.mean(vecs, axis=0)
  return sent_vec

def norm_sent_vector(sentence, w2v_model, stopword):
  vecs = [w2v_model[kata.lower()] for kata in (sentence) if kata not in stopword]
  norm_vecs = [vec / np.linalg.norm(vec) for vec in vecs if np.linalg.norm(vec) > 0]
  sent_vec = np.mean(norm_vecs, axis=0)
  return sent_vec

In [19]:
vecs = [sent_vector(kata, w2v, stop_words) for kata in sentences]
vecs = np.array(vecs)
vecs.shape

(596, 128)

## Dataset split

In [20]:
from sklearn.model_selection import train_test_split

In [36]:
X = vecs
y = df.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X_train.shape, X_test.shape

((476, 128), (120, 128))

## Training Model

In [39]:
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [40]:
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

### Naive Bayes

In [42]:
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
acc_nb = accuracy_score(y_test, y_pred_nb)
precision_nb = precision_score(y_test, y_pred_nb, average='macro')
recall_nb = recall_score(y_test, y_pred_nb, average='macro')
f1_nb = f1_score(y_test, y_pred_nb, average='macro')

In [45]:
print('Accuracy of naive Bayes classifier: {:.2f}'.format(acc_nb))
print('Precision of naive Bayes classifier: {:.2f}'.format(precision_nb))

Accuracy of naive Bayes classifier: 0.79
Precision of naive Bayes classifier: 0.73


### KNN

In [49]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_pca, y_train)
y_pred_knn = knn.predict(X_test_pca)
acc_knn = accuracy_score(y_test, y_pred_knn)

In [50]:
print('Accuracy of k-NN classifier: {:.2f}'.format(acc_knn))
print('Precision of k-NN classifier: {:.2f}'.format(acc_knn))

Accuracy of k-NN classifier: 0.77
Precision of k-NN classifier: 0.77
